In [1]:
import sys
sys.path.append('/home/dbswldms316/FLamby/')

from flamby.datasets.fed_heart_disease.dataset import HeartDiseaseRaw, FedHeartDisease

mydataset_raw = HeartDiseaseRaw()
print(mydataset_raw[0])
mydataset_pooled = FedHeartDisease(train=True, pooled=True)
print(mydataset_pooled[0])

from flamby.datasets.fed_heart_disease import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline, 
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)


(tensor([ 63.0000,   1.0000, 145.0000, 233.0000,   1.0000, 150.0000,   0.0000,
          2.3000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000]), tensor([0.]))
(tensor([ 63.0000,   1.0000, 145.0000, 233.0000,   1.0000, 150.0000,   0.0000,
          2.3000,   0.0000,   0.0000,   0.0000,   0.0000,   1.0000]), tensor([0.]))


In [2]:
import torch
# Instantiation of a list of the local test sets
test_dataset = FedHeartDisease(train=False, pooled=True)
age_groups = {}
print(test_dataset[0])
for person_idx in range(len(test_dataset)):
    person = test_dataset[person_idx]
    age = int(person[0][0])
    age_group = age - age % 10
    if age_group not in age_groups:
        age_groups[age_group] = []
    age_groups[age_group].append(person)
age_groups = dict(sorted(age_groups.items()))

result_dict = {'0 to 30': [], '40 to 60': [], 'rest':[]}
for key in age_groups.keys():
    if key <= 30:
        result_dict['0 to 30'] += age_groups[key]
    elif 40 <= key <= 60:
        result_dict['40 to 60'] += age_groups[key]
    else:
        result_dict['rest'] += age_groups[key]

print('** test dataset includes age groups like: ',[len(age_groups[i]) for i in age_groups.keys()])
print('** test dataset after split: ', [len(result_dict[i]) for i in result_dict.keys()])

# 각 나이 범주에 해당하는 데이터셋 생성
test_dataset_0_to_30 = result_dict['0 to 30']
test_dataset_40_to_100 = result_dict['40 to 60']
test_dataset_70_to_100 = result_dict['rest']

# 각 데이터셋을 DataLoader로 변환
test_dataloader_0_to_30 = torch.utils.data.DataLoader(test_dataset_0_to_30,  shuffle=False)
test_dataloader_40_to_100 = torch.utils.data.DataLoader(test_dataset_40_to_100,  shuffle=False)
test_dataloader_70_to_100 = torch.utils.data.DataLoader(test_dataset_70_to_100,  shuffle=False)
#print(next(iter(test_dataloader_0_to_30)))
# evaluate_model_on_tests 함수 호출

(tensor([ 56.0000,   1.0000, 120.0000, 236.0000,   0.0000, 178.0000,   0.0000,
          0.8000,   1.0000,   0.0000,   0.0000,   0.0000,   0.0000]), tensor([0.]))
** test dataset includes age groups like:  [19, 65, 106, 55, 9]
** test dataset after split:  [19, 226, 9]


In [3]:
import torch
# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
#feature_tensor = torch.tensor([[1,10], [2,20], [3,30], [4,40], [5,50], [6,60], [7,70], [8,80],[9,90],[10,100]])
for i in [0,1,2,3]:
    train_dataset = FedHeartDisease(center=i, train=True, pooled=False)
    print(len(train_dataset))
    #print(len(train_dataset))
    age_groups = {}
    for person_idx in range(len(train_dataset)):
        person = train_dataset[person_idx]
        age = int(person[0][0])
        age_group = int(age - age % 10)
        if age_group not in age_groups:
            age_groups[age_group] = []
        age_groups[age_group].append(person)
    age_groups = dict(sorted(age_groups.items()))
    
    result_dict = {'0 to 30':[], '40 to 60':[], '70 to 100':[]}
    for key in age_groups.keys():
        if 0 < key <=30:
            result_dict['0 to 30']+=(age_groups[key])
        elif 30 < key <= 70:
            result_dict['40 to 60']+=(age_groups[key])
        else:
            result_dict['70 to 100']+=(age_groups[key])
    print('** train dataset includes age groups like: ', [len(age_groups[i]) for i in age_groups.keys()])
    #print(result_dict)
    print('** train dataset after split: ', [len(result_dict[i]) for i in result_dict.keys()])
    # train_dataset = DictDataset(result_dict)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
    #train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    
    lossfunc = BaselineLoss()
    local_model = Baseline()
    optimizer = Optimizer(local_model.parameters(), lr=LR)
    
    
    # Traditional pytorch training loop
    for epoch in range(0, NUM_EPOCHS_POOLED):
        for X,y in train_dataloader:
            optimizer.zero_grad()
            outputs = local_model(X)
            loss = lossfunc(outputs, y)
            loss.backward()
            optimizer.step()
        
        # Evaluation
    from flamby.utils import evaluate_model_on_tests
    
    with torch.no_grad():
        dict_cindex_0_to_30 = evaluate_model_on_tests(local_model, [test_dataloader_0_to_30], metric)
        dict_cindex_40_to_60 = evaluate_model_on_tests(local_model, [test_dataloader_40_to_100], metric)
        dict_cindex_70_to_100 = evaluate_model_on_tests(local_model, [test_dataloader_70_to_100], metric)

    # 결과 출력
    print("Results for age group 0 to 39:", dict_cindex_0_to_30)
    print("Results for age group 40 to 69:", dict_cindex_40_to_60)
    print("Results for age group rest:", dict_cindex_70_to_100)


199
** train dataset includes age groups like:  [1, 11, 47, 81, 56, 3]
** train dataset after split:  [12, 187, 0]


100%|██████████| 1/1 [00:00<00:00, 749.79it/s]


Results for age group 0 to 39: {'client_test_0': 0.2631578947368421}
Results for age group 40 to 69: {'client_test_0': 0.5442477876106194}
Results for age group rest: {'client_test_0': 0.3333333333333333}
172
** train dataset includes age groups like:  [2, 30, 61, 73, 6]
** train dataset after split:  [32, 140, 0]


100%|██████████| 1/1 [00:00<00:00, 559.32it/s]


Results for age group 0 to 39: {'client_test_0': 0.6842105263157895}
Results for age group 40 to 69: {'client_test_0': 0.6858407079646017}
Results for age group rest: {'client_test_0': 0.7777777777777778}
30
** train dataset includes age groups like:  [2, 3, 11, 11, 3]
** train dataset after split:  [2, 28, 0]


100%|██████████| 1/1 [00:00<00:00, 692.93it/s]


Results for age group 0 to 39: {'client_test_0': 0.2631578947368421}
Results for age group 40 to 69: {'client_test_0': 0.5442477876106194}
Results for age group rest: {'client_test_0': 0.3333333333333333}
85
** train dataset includes age groups like:  [1, 6, 33, 37, 8]
** train dataset after split:  [1, 84, 0]


100%|██████████| 1/1 [00:00<00:00, 772.72it/s]

Results for age group 0 to 39: {'client_test_0': 0.7368421052631579}
Results for age group 40 to 69: {'client_test_0': 0.4778761061946903}
Results for age group rest: {'client_test_0': 0.3333333333333333}


In [4]:

import torch
# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer

for i in [0,1,2,3]:
    train_dataset = FedHeartDisease(center=i, train=True, pooled=False)
    #print(len(train_dataset))
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    lossfunc = BaselineLoss()
    model = Baseline()
    optimizer = Optimizer(model.parameters(), lr=LR)
    print(len(train_dataset))
    # Traditional pytorch training loop
    for epoch in range(0, NUM_EPOCHS_POOLED):
        for idx, (X, y) in enumerate(train_dataloader):
            #print(len(y))
            optimizer.zero_grad()
            outputs = model(X)
            loss = lossfunc(outputs, y)
            loss.backward()
            optimizer.step()
        
    # Evaluation
    from flamby.utils import evaluate_model_on_tests
    # Instantiation of a list of the local test sets
    test_dataloaders = [
                torch.utils.data.DataLoader(
                    FedHeartDisease(center=i, train=False, pooled=False),
                    batch_size=BATCH_SIZE,
                    shuffle=False,
                    num_workers=0,
                )
                #for i in range(NUM_CLIENTS)
            ]
    print(len(train_dataloader), len(test_dataloaders))
    # Function performing the evaluation
    dict_cindex = evaluate_model_on_tests(model, test_dataloaders, metric)
    print(dict_cindex)


199


7 1


100%|██████████| 1/1 [00:00<00:00, 260.29it/s]

{'client_test_0': 0.46153846153846156}
172


6 1


...

100%|██████████| 1/1 [00:00<00:00, 742.49it/s]

{'client_test_0': 0.9375}
85


3 1


100%|██████████| 1/1 [00:00<00:00, 430.49it/s]

{'client_test_0': 0.7111111111111111}


In [5]:
len(FedHeartDisease(center=1, train=False, pooled=False))

89